In [ ]:
from bitarray import bitarray

#Generate all possible encoded output and the space of all possible received messages for input msgs of a given length

#Jamie Taylor KK6OLF
#January 2018

#Released under CCO - have fun with it.
#WSPR encoding in Python by Jamie Taylor
#To the extent possible under law, the person who associated CC0 with
#WSPR encoding in Python has waived all copyright and related or neighboring rights to WSPR encoding in Python.
#You should have received a copy of the CC0 legalcode along with this work.  
#If not, see <http://creativecommons.org/publicdomain/zero/1.0/>.

msgbitlength = 2 #how many bits for the input message

K = 3 #constraint length
k = 1 #input bits...never used...assumed to be 1 in this demo
r = 2 #parity bits

#the taps in the convolution filter (i.e., the taps on the shift reg)
polynomials = [bitarray("111"),bitarray("101")] 

In [1]:
#sanity check the parameters
assert(len(polynomials) == r) #r must equal to the number of polynomials for parity bit generation

for poly in polynomials:
    assert(len(poly) == K) #the polynomials must be the same as the constraint length

In [2]:
def paritybits(polylist, shiftreg):
    parity = []
    
    for poly in polylist:
        tmpreg = shiftreg & poly #for each tap, see if there is a bit set in the shift register
        p = int(tmpreg.count(1) % 2) #modulo2 addition of the tap values (number of set bits mod 2)
        parity.append(p)
        
    return parity #return as many parity bits as there are polynomials describing the shiftreg taps

In [3]:
def encode(message, polylist):
    
    constraint_length = polylist[0].length()
    #create a shift register K in size, initialize to all zeros
    sr = bitarray(str("{:0"+str(constraint_length)+"b}").format(0))

    output = []
    
    #push each message bit into the shift register and generate parity bits
    for i in range(0, len(message)):
        #move the shift register one bit to the right
        for s in range(len(sr)-1, 0, -1):
            sr[s] = sr[s-1]
        #shift the next message bit into the shift register
        sr[0] = message[i]
        #print sr.to01()
        
        #generate the convolutional parity bits for this bit
        output.extend(paritybits(polylist, sr))
    return output

In [12]:
#for all possible messages of msgbitlength geneate the encoded output bits (forget tail bits)

numpossiblemsgs = 2**msgbitlength

print ""
print "all possible input and output for input messages of length " + str(msgbitlength)
print ""
print "in\tout"
print ""

for i in range(0, numpossiblemsgs):
    msg = bitarray(str("{:0"+str(msgbitlength)+"b}").format(i))
    out = encode(msg, polynomials)
    print msg.to01() + "\t" + ''.join(str(x) for x in out)


all possible input and output for input messages of length 2

in	out

00	0000
01	0011
10	1110
11	1101


In [13]:
#generate the list of all possible bit sequences that might be received for msgs of msgbitlength

#number of bits in encoded output is (msgbitlength * r) because we always assumed k to be 1 in our implementation
outputbits = msgbitlength * r

#how big is the space of possible received messages
outputspace = 2**outputbits

print ""
print "all possible receive messages of length " + str(outputbits)
print ""

for i in range(0, outputspace):
    print str("{:0"+str(outputbits)+"b}").format(i)


all possible receive messages of length 4

0000
0001
0010
0011
0100
0101
0110
0111
1000
1001
1010
1011
1100
1101
1110
1111
